# pre_Data-Format
### Step_1. FormatOfficial2Yolo

In [13]:
!python Code/pre_FormatOfficial2Yolo.py

1. Renaming Training Dataset_v5 --> Origin_Training_Dataset
2. Backing-up Labels to Dataset/Origin_Training_Dataset/origin_labels/...
100%|██████████████████████████████████████| 1000/1000 [00:01<00:00, 891.24it/s]
3. Formatting Official Labels to yolo Type...
100%|███████████████████████████████████████| 1000/1000 [00:42<00:00, 23.30it/s]
4. Moving Labels to Dataset/Origin_Training_Dataset/labels...
100%|█████████████████████████████████████| 1000/1000 [00:00<00:00, 1710.07it/s]
5. Moving images to Dataset/Origin_Training_Dataset/images...
6. Start Splitting 10% of Training Data to Val Data...
100%|████████████████████████████████████████| 100/100 [00:00<00:00, 842.31it/s]


# train_Model-Baseline
### Step_2-1. Baseline model train

In [14]:
# move the code we change in YOLOv5
!cp Code/yolov5l6.yaml yolov5/yolov5l6.yaml
!cp Code/dataset-Origin_Training_Dataset.yaml yolov5/dataset-Origin_Training_Dataset.yaml
!cp Code/hyp.none.yaml yolov5/hyp.none.yaml

# training
!python -m torch.distributed.run --nproc_per_node 2 yolov5/train.py --weights yolov5/yolov5l6.pt --cfg yolov5/yolov5l6.yaml --hyp yolov5/hyp.none.yaml --data yolov5/dataset-Origin_Training_Dataset.yaml --epochs 1000 --batch-size 8 --imgsz 1920 --device 0,1 --patience 50

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Unknown option: -C
usage: git [--version] [--help] [-c name=value]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p|--paginate|--no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           <command> [<args>]
train: weights=yolov5/yolov5l6.pt, cfg=yolov5/yolov5l6.yaml, data=yolov5/dataset-Origin_Training_Dataset.yaml, hyp=yolov5/hyp.none.yaml, epochs=1000, batch_size=8, imgsz=1920, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=0,1, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=F

### Step_2-2. Baseline model detect (Public 0.70)

In [ ]:
!python yolov5/detect.py --data yolov5/dataset-Origin_Training_Dataset.yaml --imgsz 1080 1920 --save-txt --save-conf --weight yolov5/runs/train/exp/weights/best.pt --source Dataset/public
# If you skip training you can simply run the following line
# !python yolov5/detect.py --data yolov5/dataset-Origin_Training_Dataset.yaml --imgsz 1080 1920 --save-txt --save-conf --weight Dataset/Models/baseline.pt --source Dataset/public

In [ ]:
!python Code/post_FormatYolo2Official.py

# pre_Data-Cleaning
### Step_3. Analyze bbox
### Mistaken bbox

In [ ]:
!python Code/pre_AnalyzeBBox_Mistaken.py

import cv2
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(cv2.imread('Output/AnalyzeBBox_Mistaken.png'))

### Outlier bbox


In [ ]:
!python Code/pre_AnalyzeBBox_Outlier.py

# pre_Data-Classify
### Step_4. Classify bbox with size
### K-means

In [ ]:
!python Code/pre_AnalyzeBBox_kmeans.py

import matplotlib.pyplot as plt
Classes = ['car', 'hov', 'person', 'motorcycle']

for Class in Classes:
    plt.figure(figsize=(5, 5))
    plt.axis('off')
    plt.imshow(cv2.imread(f'Output/AnalyzeBBox_3-means_{Class}.png'))

# pre_Data-Cleaning
### Step_5. Compare GT bbox with baseline prediction

In [ ]:
# Just simply download the labels we already re-labeled.

# How we re-labeled:
# 1. Detect training dataset with baseline.pt
# 2. Preserve the prediction's boundingboxes which have an IoU > 0.5 boundingbox in origin training dataset
#    (why? Because the prediction boundingboxes' quality is better than human-labelling, which can avoid being insufficient or exceed the object seriously.)
# 3. Compare remaining prediction's boundingboxes one-by-one with my human eye

# pre_Data-SuperResolution
### Step_6. Resize images 2 times larger


In [ ]:
# Just simply download the images we super-resolved.

### Step_7. Resize all images to 3840x2160 with Bicubic

In [ ]:
!python pre_ResizeSuperResolution.py

# pre_Data-Cleaning
### Step_8. Filter out the ignored areas

In [ ]:
!python pre_FilterIgnoredAreas.py

# pre_Data-Slicing
### Step_9. Slice Super Resolution images (3840x2160 --> 832x832)

In [ ]:
# Create COCO format for slicing
!python pre_FormatYolo2Coco.py  --path Dataset/SuperResolution_Training_Dataset

!pytohn pre_Slicing.py

# pre_Data-Augmentation
### Step_11. Augmentation

In [ ]:
# move the code we change in MMDetection
!cp Code/mm__init__.py mmdetection/mmdet/datasets/pipelines/__init__.py
!cp Code/browse_dataset.py mmdetection/tools/misc/browse_dataset.py
!cp Code/image.py mmdetection/mmdet/core/visualization/image.py
!cp Code/SmallObjectAugmentation.py mmdetection/mmdet/datasets/pipelines/SmallObjectAugmentation.py
!cp Code/transform.py mmdetection/ mmdetection/mmdet/datasets/pipelines/transform.py
!cp Code/dataset_visualize.py mmdetection/dataset_visualize.py

# install mmdetection
!pip install -U openmim
!mim install mmcv-full
# !cd mmdetection
!pip install -v -e mmdetcttion


import os

if not os.path.exists('Dataset/SuperResolution_Training_Sliced_Augmentation'):
    os.mkdir('Dataset/SuperResolution_Training_Sliced_Augmentation')


!python mmdetcttion/tools/misc/browse_dataset.py mmdetcttion/dataset_visualize.py --output-dir Dataset/SuperResolution_Training_Sliced_Augmentation/ --not-show

# train_Model-832x832
### Step_12-1. 832x832 model train


In [ ]:
import random
from tqdm import tqdm

print('Start Splitting 10% of Training Data to Val Data...')

TrainingDatasetRoot = 'Dataset/SuperResolution_Training_Sliced_Augmentation'
os.mkdir(f'{TrainingDatasetRoot}images')
os.mkdir(f'{TrainingDatasetRoot}labels')
os.mkdir(f'{TrainingDatasetRoot}images/train')
os.mkdir(f'{TrainingDatasetRoot}labels/train')
os.mkdir(f'{TrainingDatasetRoot}images/val')
os.mkdir(f'{TrainingDatasetRoot}labels/val')
val_index = random.sample(range(1, 1000), 100)
for index in tqdm(val_index):
    os.rename(f'{TrainingDatasetRoot}images/train/img{index:04d}.png',
              f'{TrainingDatasetRoot}images/val/img{index:04d}.png')
    os.rename(f'{TrainingDatasetRoot}labels/train/img{index:04d}.txt',
              f'{TrainingDatasetRoot}labels/val/img{index:04d}.txt')


#training
!python -m torch.distributed.run --nproc_per_node 2 yolov5/train.py --weights yolov5/yolov5l6.pt --cfg yolov5/yolov5l6.yaml --hyp yolov5/hyp.high_custom.yaml --data yolov5/dataset-SuperResolution_Training_Sliced_Augmentation.yaml --epochs 1000 --batch-size 20 --imgsz 1664 --device 0,1 --patience 50

### Step_12-2. 832x832 model detect (Public 0.724934 / Private 0.753749)

In [ ]:
!cp -R Dataset/SuperResolution_Public_Dataset/*.png Dataset/SuperResolution_PublicPrivate_Dataset/

!python yolov5/detect.py --data yolov5/dataset-SuperResolution_Training_Sliced_Augmentation.yaml --imgsz 3072 5376 --save-txt --save-conf --weight yolov5/runs/train/exp2/weights/best.pt --source Dataset/SuperResolution_PublicPrivate_Dataset --augment --conf-thres 0.4
# If you skip training you can simply run the following line
# !python yolov5/detect.py --data yolov5/dataset-SuperResolution_Training_Sliced_Augmentation.yaml --imgsz 3072 5376 --save-txt --save-conf --weight Dataset/Models/832x832.pt --source Dataset/SuperResolution_PublicPrivate_Dataset --augment --conf-thres 0.4

In [ ]:
!python Code/post_FormatYolo2Official.py

### Step_12-2. 832x832 model detect WBF with Baseline model (Public 0.725854 / Private 0.754987)
# This is a mysterious step, but it made the highest score !


In [ ]:
# Detect the super-resolved public and private images with baseline model first
!python yolov5/detect.py --data yolov5/dataset-Origin_Training_Dataset.yaml --imgsz 3072 5376 --save-txt --save-conf --weight yolov5/runs/train/exp/weights/best.pt --source Dataset/SuperResolution_PublicPrivate_Dataset --augment --conf-thres 0.4
# If you skip training you can simply run the following line
# !python yolov5/detect.py --data yolov5/dataset-Origin_Training_Dataset.yaml --imgsz 3072 5376 --save-txt --save-conf --weight Dataset/Models/baseline.pt --source Dataset/SuperResolution_PublicPrivate_Dataset --augment --conf-thres 0.4

In [ ]:
# WBF the result of 832x832 model and baseline model with weights of 1.3 : 1
!python Code/post_WBF.py

In [ ]:
!python Code/post_FormatYolo2Official.py
